# Logging

Python has a standard library for logging: `logging`.

- [Logging in python: developer guide (rus)](https://habr.com/ru/companies/wunderfund/articles/683880/);
- ["Logging HOWTO"](https://docs.python.org/3/howto/logging.html#configuring-logging) from docs.python.org contains two tutorials: basic and advanced.

In [1]:
import sys
import logging

The core components of the `logging` library are:

- Loggers
- Handlers
- Formatters
- Filters

Each one defines a specific aspect of log generation.

## Loggers

For different parts of your program, you may need separate `loggers`. You can create a new logger by using `logging.getLogger(<logger_name>)`. Find out more details in the [specific page](logging/loggers.ipynb).

---

The following cell demonstrates the creation of `logger1` and `logger2`, along with their immediate usage.

In [2]:
logger1 = logging.getLogger("logger1")
logger1.setLevel(logging.INFO)
print("=====logger1=====", file=sys.stderr)
logger1.debug("A DEBUG Message")
logger1.info("An INFO")
logger1.warning("A WARNING")
logger1.error("An ERROR")
logger1.critical("A message of CRITICAL severity")


logger2 = logging.getLogger("logger2")
logger2.setLevel(logging.ERROR)
print("=====logger2=====", file=sys.stderr)
logger2.debug("A DEBUG Message")
logger2.info("An INFO")
logger2.warning("A WARNING")
logger2.error("An ERROR")
logger2.critical("A message of CRITICAL severity")

=====logger1=====
A WARNING
An ERROR
A message of CRITICAL severity
=====logger2=====
An ERROR
A message of CRITICAL severity


## Handlers

Defines the direction in which logs are written. It can be files or output streams. You can add handlers to the logger using the `logger.addHandler` method. Check list of the [usefull handlers](https://docs.python.org/3/howto/logging.html#useful-handlers).

---

The following example demonstrates how to add a handler to a logger.

In [3]:
logger = logging.getLogger("temp_logger")
stream_handler = logging.StreamHandler()
logger.addHandler(stream_handler)
logger.critical("It's a error!")
# to prevent adding a handler for each
# run of this cell, we'll delete just
# added logger
del logger.handlers[0]

It's a error!


There is a [specfic page describing handlers](./logging/handler.ipynb).

## Formatters

Formatters define how each line of the log is to be printed. You must set the formatter for the handler using the `handler.setFormatter` method.

---

The following example defines formatter for given logger.

In [4]:
logger = logging.getLogger("some_logger")
stream_handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.error("It's error")
del logger.handlers[0]

2024-08-29 15:15:36,721 - some_logger - ERROR - It's error


## Configuration

There are some tools that allows to configure `logging` module from optional `key:value` like formats.

There is [special page](logging/configuration.ipynb) about aspects of working with it.

---

Here's a simple example of how to configure an entire logging module using Python dictionaries. In this example, a logger named `simpleExample` is created with a specific formatter to ensure the logs have the desired format.

In [5]:
logging.config.dictConfig({
    "version" : 1,
    "formatters" : {
        "simpleFormatter" : {
            "format" : "I'm created from config :) %(message)s"
        }
    },
    "handlers" : {
        "consoleHandler" : {
            "class" : "logging.StreamHandler",
            "formatter" : "simpleFormatter"
        }
    },
    "loggers" : {
        "simpleExample" : {
            "handlers" : ["consoleHandler"]
        }
    }
})

logging.getLogger("simpleExample").error("{it's message}")

I'm created from config :) {it's message}


It is the preferred way to configure the logs, because anyone who is not involved with the code can participate in configuring the logs.

## Root logger

This logger is used when you run the logging methods of the `logging` module itself.

Using the `logging.basicConfig` function, you can define the behaviour of the logs generated by the `logging` module. The following example defines the `DEBUG` logging level so that you can later see any message raised by the `logging` methods. By default only `warning`, `error` and `critical` are executed.

**Note** In the following example, the code is saving the logs to a separate file and executing it from another interpreter. This approach is necessary because once you execute any logging method in this notebook, you cannot modify its configuration anymore.

In [6]:
%%writefile logging_files/basic_logger.py
import logging
logging.basicConfig(level=logging.DEBUG)

logging.debug("A DEBUG Message")
logging.info("An INFO")
logging.warning("A WARNING")
logging.error("An ERROR")
logging.critical("A message of CRITICAL severity")

Overwriting logging_files/basic_logger.py


In [7]:
!python3 logging_files/basic_logger.py

DEBUG:root:A DEBUG Message
INFO:root:An INFO
ERROR:root:An ERROR
CRITICAL:root:A message of CRITICAL severity


### Format

To set up the format for the basic logger, you should use the `format` parameter of the `logging.basicConfig` function.

The following example sets up basic logger to print the time of the event and it's message.

In [8]:
%%writefile logging_files/basic_logger.py
import logging
logging.basicConfig(format='%(asctime)s %(message)s')

logging.debug("A DEBUG Message")
logging.info("An INFO")
logging.warning("A WARNING")
logging.error("An ERROR")
logging.critical("A message of CRITICAL severity")

Overwriting logging_files/basic_logger.py


In [9]:
!python3 logging_files/basic_logger.py

2024-08-29 15:17:28,681 A WARNING
2024-08-29 15:17:28,681 An ERROR
2024-08-29 15:17:28,681 A message of CRITICAL severity


To check format of the basic logger you need:

- Use `logging.getLogger()` to get the basic logger;
- Grab the handler you're interested in. By default, basic loggers have only one handler;
- The `formatter._fmt` field of the handler would be a string defining the format.

The following cell prints the notebooks handler format.

In [28]:
logging.basicConfig(format="%(levelname)s => %(message)s")
basic_logger = logging.getLogger()
print(basic_logger.handlers[0].formatter._fmt)
logging.warning("test")

%(levelname)s:%(message)s
